In [7]:
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt
from uk_covid19 import Cov19API
import datetime
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [8]:
# Steps done within this function
# 1 - convert data type of 'dates' column from string to datetime
# 2 - make 'dates' the new index (old index is just 0, 1, ...)
# 3 - sort in ascending order of dates
# 4 - replace missing data using linear interpolation
def wrangle_deaths_data(df):
    df['dates'] = pd.to_datetime(df['dates'], format="%Y-%m-%d")
    df.set_index('dates', drop=True, append=False, inplace=True)
    df.sort_index(ascending=True, inplace=True)
    df.interpolate(method='linear', axis=0, limit_direction = 'both', inplace=True)

nations=['England', 'Scotland', 'Wales', 'Northern Ireland']
supportedMetrics = ['deaths', 'hospitalCases', 'firstDose', 'secondDose', 'thirdDose']

# Steps done within this function
# 1 - constructs 'filters' for a given nation ('England', ...)
# 2 - constructs 'structure' to obtain dates and predefined metrics
# 3 - creates an instance of SDK API
# 4 - makes a remote call to fetch actual data as a panda DataFrame object
def access_api(nation):
    filters = ['areaType=nation', 'areaName={}'.format(nation)]
    structure = {'dates': 'date',
                 'deaths': 'cumDeaths28DaysByDeathDateRate',
                 'hospitalCases':'hospitalCases',
                 'firstDose' : 'cumPeopleVaccinatedFirstDoseByPublishDate',
                 'secondDose' : 'cumPeopleVaccinatedSecondDoseByPublishDate',
                 'thirdDose':'cumPeopleVaccinatedThirdInjectionByPublishDate'
                }
    try:
        # no internet connection situation is handled by the SDK and kernel.
        # catching other exceptions here for extra safety
        api = Cov19API(filters=filters, structure=structure)
        df = api.get_dataframe()
        return df
    except:
        pass
    
dfAllNations = pd.DataFrame()
try:
    dfAllNations = pd.read_json('allData.json')
except:
    print('Could not open file')

# dummyCheckbox is only to implement force replotting after refresh data
# is comeplete. Using dummyCheckbox instead of actual widget is preferrable
# since we can implement re-plotting WITHOUT actually displaying the checkbox
# hence avoiding any visual glitch.
dummyCheckbox = wdg.Checkbox(
    value=False,
    description='Dummy',
    disabled=False,
    indent=False
)

# this function changes the value of the dummyCheckbox
# to the opposite value thus forcing re-plotting. The actual value of checkbox
# is not important which is why we don't set it back.
# What matters is that there is a change which can trigger further actions
def force_replot():
    global nationsList
    dummyCheckbox.value = not dummyCheckbox.value

# Steps done within this function
# 1 - loops over all nations
# 2 - makes an API call to fetch a fresh data for the given nation
# 3 - wrangles fetched data and adds to the local dictionary
# 4 - merges all 4 dataframes into 1 which contains data for all nations
# 5 - replace missing data using linear interpolation (it is possible after merge
#     when indexes of 2 different dataframes are different

def refresh_data(button):
    global dfAllNations
    dfs = {}
    for nation in nations:
        df = access_api(nation)
        wrangle_deaths_data(df)
        dfs[nation] = df
    
    
    d = {f'{nation}_{metric}' : dfs[nation][metric] for metric in supportedMetrics for nation in nations}
    dfAllNations = pd.DataFrame(d)
    dfAllNations.interpolate(method='linear', axis=0, limit_direction = 'both', inplace=True)
    force_replot()

## COVID dashboard of UK nations based on different metrics
This dashboard uses a few filters which can be used to customise the data being displayed. Currently available filters are start and end dates,
nations list and metrics. After changing any of those filters the graph auto-refreshes to display data accordingly.
The scale radio button does not impact the data except that it displays graphs using a logarithmic scale for the y-axis.
The refresh data button when clicked will fetch and display the latest data from PHE.
The dashboard can be used offline as well. The only time it requires an Internet connection is when the refresh button is clicked.

This dashboard is using Public Health England(PHE) data. The link can be found [here](https://coronavirus.data.gov.uk/)

In [9]:

refreshButton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='info', 
    tooltip='Click to refresh with current data',
    icon='success')

refreshButton.on_click(refresh_data)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

nationsList = wdg.SelectMultiple(
    options=nations,
    value=nations[0:2],
    description='Nation',
    disabled=False
)

startDate = wdg.DatePicker(
    description='Start Date',
    value = datetime.date(2020,3,1),
    disabled=False
)
endDate = wdg.DatePicker(
    description='End Date',
    value = datetime.date.today(),
    disabled=False
)

metrics=wdg.Dropdown(
    options=supportedMetrics,
    value=supportedMetrics[0],
    description='Metrics:',
    disabled=False,
)
# Steps done within this function
# reset start and end dates if date range isn't valid
# by default legend matches DataFrame colmn names.
# the idea of how to change it is taked from https://stackoverflow.com/a/33150133
#plot the data
def plotDeathsData(nations, dummyBool, scale, start, end, metric):
    global startDate, endDate
    if end - start <= datetime.timedelta(days=1):
        startDate.value = datetime.date(2020,3,1)
        endDate.value = datetime.date.today()
        return
    try:
        columns = [f"{nation}_{metric}" for nation in nations]
        if not dfAllNations.empty:
            fig, ax = plt.subplots()
            dfAllNations[start:end][columns].plot(logy=(scale=='log'), xlabel="", ax=ax, ylabel=metric)
            ax.legend(nations);
            plt.show()
    except:
        pass
        
out = wdg.interactive_output(plotDeathsData,
                             {'nations' : nationsList,
                              'dummyBool' : dummyCheckbox,
                              'scale': scale,
                              'start': startDate,
                              'end': endDate,
                              'metric': metrics})


hb1 = wdg.HBox([startDate, endDate])
vb1 = wdg.VBox([nationsList, metrics, scale])
hb2 = wdg.HBox([out, vb1])
layout=wdg.VBox([hb1, hb2, refreshButton])

display(layout)